# Download Data

In [1]:
import requests
import pandas as pd
from io import BytesIO

def download_csv(csv_url):
    response = requests.get(csv_url)
    response.raise_for_status()

    # Qui puoi modificare 'sep' o aggiungere altri parametri se necessario
    try:
        df = pd.read_csv(BytesIO(response.content), sep=',')
    except pd.errors.ParserError:
        df = pd.read_csv(BytesIO(response.content), sep=';', error_bad_lines=False)

    return df

In [2]:
recipes_df = download_csv("https://media.githubusercontent.com/media/DonatoFe11/SustainaMeal_Case_Study/master/data/final_recipes_set.csv")

In [3]:
recipes_df

,title,sustainability_score,sustainability_label,Unnamed: 0,recipe_id,description,author_id,duration,directions,ingredients,...,protein [g],direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,normalization_comment,ingredient_food_kg_urls,ingredient_food_kg_names,healthiness_label
0,Boiled Radishes,0.002502,0,294268,179840,These taste a lot like new potatoes. The boil...,254469,22.0,"['Trim ends and bad spots off Radishes.', 'Boi...","['radishes size bag bunch', 'water radishes']",...,0.8,2,2,0.293040,1.000,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['applesauce', 'garlic clove', 'ginger', 'mola...",0
1,Apple Cider Reduction,0.002623,0,59722,521756,I saw a post about this on Facebook and gave i...,171084,155.0,"['In 6-8 quart stock pan, measure out 2 cups o...","['gallon apple cider fresh', 'gallon apple cid...",...,0.0,10,2,0.214286,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['bean sprouts', 'chicken thighs', 'fresh ging...",1
2,Homemade Apple Pectin Stock,0.002623,0,159910,25188,Easy to make and works well.,10404,25.0,['Slice unpeeled apples including cores and se...,"['tart apple', 'water pound apples']",...,1.2,11,2,0.278414,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['baking powder', 'baking soda', 'cinnamon', '...",0
3,Easy Applesauce,0.002623,0,421226,264179,Use fresh cooking apples for this for best res...,37636,50.0,['Place apples and cider into a large saucepan...,"['apples', 'apple cider']",...,0.5,3,2,0.276525,0.875,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['carrots', 'fresh peas', 'fresh tomatoes', 'g...",0
4,Cider Baked Sausage,0.002752,0,374088,101966,A very easy to make main dish perfect for fall...,30534,50.0,"['Preheat oven to 350°.', 'Prick sausages with...","['breakfast sausage links', 'apples Smith', 'o...",...,46.4,4,4,0.146714,0.125,0.00,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Shedd Spread Country Crock', 'cayenne pepper...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93743,Curried Lamb on Rice,0.951024,2,103145,373244,"With modification this recipe is from ""The Dia...",133174,65.0,"['Over medium heat, melt the butter in a large...","['lamb', 'medium onion', 'cloves', 'butter', '...",...,28.7,5,13,0.223549,0.625,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Dijon mustard', 'Polish sausage', 'apricot p...",1
93744,Rich Lamb Curry,0.960504,2,442694,268247,The recipe isn't as time consuming as it looks...,491437,110.0,"['Heat 1 tbsp olive oil in dutch oven. ', 'Co...","['oil', 'lamb', 'flour', 'salt pepper', 'onion...",...,36.1,9,21,0.153328,0.000,0.00,NaN,NaN,NaN,1
93745,"Middle Eastern Slow-Cooked Stew With Lamb, Chi...",0.968458,2,359960,144850,From Cooking Light. Per 3/4 c. serving: 310 ca...,37779,112.0,['Let the oil get heating in a large pot over ...,"['oil', 'lamb', 'onions rings', 'water', 'clov...",...,20.8,12,19,0.205566,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['anchovies', 'celery ribs', 'diced tomatoes',...",1
93746,Five Meat Chili Con Carne With Beans,0.971454,2,37637,13568,yum,20571,210.0,"['In a large pot brown meat in stages, about a...","['beef', 'pork', 'lamb', 'sausage', 'sausage',...",...,14.5,7,15,0.197568,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"[""French\\'s French fried onions"", 'crabmeat',...",1


# SustainaMeal (ETA: 15/20 mins)

Initializes the system by loading the data and preparing the embeddings.

In [4]:
from HeASe.sustainameal import SustainaMeal

nutrients = ['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]']

sm = SustainaMeal(
    recipes_df=recipes_df,
    nutrients=['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]'],
    transformer_name='davanstrien/autotrain-recipes-2451975973'
)

/home/festa/SustainaMeal_Case_Study/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing Titles embeddings: 100%|██████████| 93748/93748 [14:24<00:00, 108.49batch/s]


# Upload Recipes of experiments

In [5]:
import pandas as pd

# Paths to the JSON files

sustainability_label_2_recipes_file = '../experiments/data/sustainability_label_2_recipes.json'

# Read the JSON files into pandas DataFrames

sustainability_label_2_recipes = pd.read_json(sustainability_label_2_recipes_file)


# Compare LLM Rerank

In [6]:
sm.setup_openai_key("opken-ai")

In [7]:
acceptable_tags = ['appetizers', 'main-dish', 'side-dishes', 'drinks', 'beverages', 'fruits', 'desserts',
                   'breakfast', 'pasta-rice-and-grains', 'rice', 'pasta', 'pizza', 'breads', 'meat', 'fish',
                   'seafood', 'beef', 'chicken', 'vegetarian']

In [8]:
nutrients_string = ", ".join(nutrients)
print(nutrients_string)


calories [cal], totalFat [g], saturatedFat [g], cholesterol [mg], sodium [mg], dietaryFiber [g], sugars [g], protein [g]


In [9]:
selected_recipes = pd.concat([sustainability_label_2_recipes])

In [10]:
len(selected_recipes)

100

In [11]:
# Definisci le soglie
WHO_SCORE_THRESHOLDS = ( 0.15, 0.25)  # (soglia media, soglia alta)
SUSTAINABILITY_SCORE_THRESHOLDS = (0.1, 0.5)  # (soglia media, soglia alta)

# Funzioni di aiuto per calcolare le label
def calculate_healthiness_label(who_score):
    if who_score > WHO_SCORE_THRESHOLDS[1]:
        return "0"
    elif who_score < WHO_SCORE_THRESHOLDS[0]:
        return "2"
    else:
        return "1"

def calculate_sustainability_label(sustainability_score):
    if sustainability_score <= SUSTAINABILITY_SCORE_THRESHOLDS[1]:
        return "0"
    elif sustainability_score >= SUSTAINABILITY_SCORE_THRESHOLDS[1]:
        return "2"
    else:
        return "1"

# Calcola le label in base ai valori di who_score e sustainability_score

In [12]:
similar_recipes = sm.find_similar_recipes("Pasta", 10, acceptable_tags=acceptable_tags, match_all_tags=False)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 44.05batch/s]

Tags to match: ['main-dish', 'pasta', 'vegetarian', 'pasta-rice-and-grains']
Number of filtered recipes: 40632


In [13]:
ordered_recipes = sm.order_recipe_by_sustainameal()
print(ordered_recipes)

                                                   title  similarity_score  \
14718                             Brown Rice and Edamame          0.983478   
15242  Tofu With Red Curry Paste, Peas, and Yellow To...          0.987000   
10896                Pasta with Tomatoes, Peas and Cream          0.989872   
27313                                          Tofu Bowl          0.982967   
47161                       Asparagus-Mushroom Primavera          0.989029   
59687         Angel Hair With Tomatoes, Basil and Garlic          0.983265   
65588                  Spaghettini With Smothered Onions          0.983291   
60293                    Orlando Bloom's Pasta Au Pistou          0.985503   
68061               Mini Farfalle With Tomatoes and Corn          0.991634   
79355                Pasta and Potatoes (Pasta e Patate)          0.984892   

       who_score  healthiness_increment  sustainability_score  \
14718   0.345328              47.711098              0.909384   
15242   0.3

In [14]:
ordered_recipes_gpt = sm.order_recipe_by_sustainameal()
print(ordered_recipes_gpt)

                                                   title  similarity_score  \
14718                             Brown Rice and Edamame          0.983478   
15242  Tofu With Red Curry Paste, Peas, and Yellow To...          0.987000   
10896                Pasta with Tomatoes, Peas and Cream          0.989872   
27313                                          Tofu Bowl          0.982967   
47161                       Asparagus-Mushroom Primavera          0.989029   
59687         Angel Hair With Tomatoes, Basil and Garlic          0.983265   
65588                  Spaghettini With Smothered Onions          0.983291   
60293                    Orlando Bloom's Pasta Au Pistou          0.985503   
68061               Mini Farfalle With Tomatoes and Corn          0.991634   
79355                Pasta and Potatoes (Pasta e Patate)          0.984892   

       who_score  healthiness_increment  sustainability_score  \
14718   0.345328              47.711098              0.909384   
15242   0.3

In [15]:
best_choice_by_gpt = sm.choose_best_recipe_with_gpt()

1. Recipe: Brown Rice and Edamame
2. Recipe: Tofu With Red Curry Paste, Peas, and Yellow Tomatoes
3. Recipe: Asparagus-Mushroom Primavera
4. Recipe: Tofu Bowl
5. Recipe: Mini Farfalle With Tomatoes and Corn
6. Recipe: Pasta with Tomatoes, Peas and Cream
7. Recipe: Angel Hair With Tomatoes, Basil and Garlic
8. Recipe: Spaghettini With Smothered Onions
9. Recipe: Pasta and Potatoes (Pasta e Patate)
10. Recipe: Orlando Bloom's Pasta Au Pistou

Choose: Brown Rice and Edamame


In [16]:
import json
import pandas as pd

# Assumi che 'SustainaMeal', 'acceptable_tags', 'nutrients_string', e 'recipe_titles' siano definiti

# Parametri per gli esperimenti
k_values = [10]
match_all_tags_values = [False]

# Funzione ausiliaria per convertire oggetti in formati serializzabili in JSON
def convert_to_serializable(obj):
    if hasattr(obj, 'to_dict'):
        return obj.to_dict('records')
    elif hasattr(obj, 'tolist'):
        return obj.tolist()
    elif isinstance(obj, (list, dict, str, int, float, bool, type(None))):
        return obj
    else:
        return str(obj)

results = []  # Lista per raccogliere i risultati degli esperimenti

for index, row in selected_recipes.iterrows():
    input_recipe = row['title']
    match_results = {}  # Dizionario per memorizzare i risultati di match per ogni k
    for k in k_values:
        match_results[k] = {'exact_match': False, 'match_count': 0}  # Inizializza il conteggio e il match esatto per ogni k
        for match_all_tags in match_all_tags_values:
            try:
                similar_recipes = sm.find_similar_recipes(input_recipe, k, acceptable_tags=acceptable_tags, match_all_tags=match_all_tags)
                ordered_recipes = sm.order_recipe_by_sustainameal()
                ordered_recipes_gpt = sm.order_recipe_by_sustainameal()

                best_choice_by_gpt = sm.choose_best_recipe_with_gpt()
                gpt_choice_match = ordered_recipes.iloc[0]['title'] == best_choice_by_gpt
                if gpt_choice_match:
                    match_results[k]['match_count'] += 1
                    if match_all_tags:  # Se consideri importante il match quando match_all_tags è True
                        match_results[k]['exact_match'] = True
                result = {
                    'recipe_name': input_recipe,
                    'recipe_id': row['recipe_id'],
                    'sustnability_score': row['sustainability_score'],
                    'sustnability_label': row['sustainability_label'],
                    'who_score': row['who_score'],
                    'healthiness_label': row['healthiness_label'],
                    'k': k,
                    'nutrients': nutrients_string,
                    'match_all_tags': match_all_tags,
                    'gpt_choice': best_choice_by_gpt,
                    'similar_recipes': convert_to_serializable(similar_recipes) if similar_recipes is not None else None,
                    'ordered_by_sustainameal': convert_to_serializable(ordered_recipes) if ordered_recipes is not None else None
                }
                results.append(result)
            except Exception as e:
                print(f"An error occurred while finding similar recipes: {e}")
                best_choice_by_gpt = None



    # Calcola la percentuale di similarità per ogni k e aggiungi ai risultati
    # for k, match_info in match_results.items():
    #     match_info['similarity_percentage'] = (match_info['match_count'] / len(match_all_tags_values)) * 100
    #     results.append({
    #         'recipe_name': input_recipe,
    #         'k': k,
    #         'exact_match': match_info['exact_match'],
    #         'similarity_percentage': match_info['similarity_percentage']
    #     })

# Salva i risultati in un file JSON
with open('experiment_results_with_gpt_and_similarity_k_10.json', 'w') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 61.02batch/s]

Tags to match: ['main-dish', 'beef', 'meat']
Number of filtered recipes: 28937
Based on a balance of sustainability and healthiness, the most recommended recipe is "Italian Chicken and Artichokes."


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 71.80batch/s]


Tags to match: ['main-dish', 'meat', 'pasta-rice-and-grains']
Number of filtered recipes: 34344
1. Recipe: Delicious Vegetable Lasagna
2. Recipe: Linguine With Marinara Sauce and Meatballs
3. Recipe: Delicious Game Day Beef Stew
4. Recipe: Amager Island Stew (Amagergryde)
5. Recipe: Bewitching Chili
6. Recipe: Hamburger Hot Pot (crock Pot)
7. Recipe: Asparagus, Cabbage and Chicken or Chicken Stir-Fry
8. Recipe: Easy & Yummy Slow Cooker Chicken
9. Recipe: Quick Meatball Lasagna
10. Recipe: Mandarin Pork

You should choose: Recipe: Delicious Vegetable Lasagna


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 97.68batch/s]


...

Tags to match: ['main-dish', 'beef', 'meat']
Number of filtered recipes: 28937
1. Recipe: Easy Chicken Almondine
2. Recipe: Alice's Baked Chicken Breast
3. Recipe: Baked Chicken and Wine
4. Recipe: Clahan's Jersey Beef
5. Recipe: Beef Dinner
6. Recipe: Pastitsio (Better Than Spaghetti)
7. Recipe: Pollo Fundido
8. Recipe: Linda's Tagborinni
9. Recipe: Simply Delicious Lasagna
10. Recipe: Creamy Cheesy Lobster Cavatappi

Choose "Easy Chicken Almondine".


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 79.41batch/s]


Tags to match: ['main-dish', 'pasta', 'pasta-rice-and-grains']
Number of filtered recipes: 30436
Here is the recommended recipe based on a balance of sustainability and healthiness:

93259. Recipe: Easy Crock Pot Lasagna


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 59.83batch/s]


Tags to match: ['main-dish', 'beef', 'rice', 'meat', 'pasta-rice-and-grains']
Number of filtered recipes: 34344
1. Sensational Spinach Stuffed Shells
2. Beef Lombardi
3. Stuffed Bell Pepper Stew
4. Almost Taco Bell Famous Tacos (In Our House!!)
5. Chicken Soup With Gouda
6. Yum Nua - Thai Beef Salad
7. Wor Won Ton Soup
8. Cheesy Ham and Sweet Potato Casserole
9. Quick Ham and Potato Scallop
10. Rice and Ham Casserole

Choose: Sensational Spinach Stuffed Shells


In [17]:
len(results)

100

In [18]:
import json
import pandas as pd

# Carica i dati dal file JSON
with open('experiment_results_with_gpt_and_similarity_k_10.json', 'r') as file:
    experiment_data = json.load(file)

# Converti i dati in un DataFrame
experiment_df = pd.DataFrame(experiment_data)

# Assumiamo di avere gli ID delle ricette per ogni sottogruppo
sustainability_label_2_ids = sustainability_label_2_recipes['recipe_id'].unique()


In [19]:
import json
import pandas as pd

# Carica i dati dal file JSON
with open('experiment_results_with_gpt_and_similarity_k_10.json', 'r') as file:
    experiment_data = json.load(file)

# Converti i dati in un DataFrame
experiment_df = pd.DataFrame(experiment_data)

# Estrai solo le entrate dove GPT ha fatto una scelta
gpt_choices_df = experiment_df.dropna(subset=['gpt_choice'])

# Inizializza i contatori per gli incrementi
healthiness_increments = []
sustainability_increments = []
sustainameal_score_increments = []

# Calcola gli incrementi solo per le scelte di GPT
for index, row in gpt_choices_df.iterrows():
    for ordered_recipe in row['ordered_by_sustainameal']:
        if ordered_recipe['title'] == row['gpt_choice']:
            healthiness_increments.append(ordered_recipe['healthiness_increment'])
            sustainability_increments.append(ordered_recipe['sustainability_increment'])
            sustainameal_score_increments.append(ordered_recipe['sustainameal_score_increment'])
            break  # Interrompi il ciclo dopo aver trovato la corrispondenza

# Calcola le medie degli incrementi per le scelte di GPT
average_healthiness_increment = sum(healthiness_increments) / len(healthiness_increments) if healthiness_increments else None
average_sustainability_increment = sum(sustainability_increments) / len(sustainability_increments) if sustainability_increments else None
average_sustainameal_score_increment = sum(sustainameal_score_increments) / len(sustainameal_score_increments) if sustainameal_score_increments else None

# Crea un DataFrame con una riga contenente le medie e la colonna gpt_rerank impostata a True
gpt_results_df = pd.DataFrame({
    'top_healthiness_increment_mean': [average_healthiness_increment],
    'top_sustainability_increment_mean': [average_sustainability_increment],
    'top_sustainameal_score_increment_mean': [average_sustainameal_score_increment],
    'gpt_rerank': [True],
    'k':10,
})

# Stampa il DataFrame dei risultati
print(gpt_results_df)


   top_healthiness_increment_mean  top_sustainability_increment_mean  \
0                       15.833658                          90.176623   

   top_sustainameal_score_increment_mean  gpt_rerank   k  
0                              72.819732        True  10  


In [20]:
import json
import pandas as pd

# Carica i dati dal file JSON
with open('experiment_results_with_gpt_and_similarity_k_10.json', 'r') as file:
    experiment_data = json.load(file)

# Converti i dati in un DataFrame
experiment_df = pd.DataFrame(experiment_data)

# Inizializza i contatori per gli incrementi
top_healthiness_increments = []
top_sustainability_increments = []
top_sustainameal_score_increments = []

# Calcola gli incrementi per le prime ricette in ordered_by_sustainameal
for index, row in experiment_df.iterrows():
    if row['ordered_by_sustainameal']:  # Controlla se la lista non è vuota
        top_recipe = row['ordered_by_sustainameal'][0]  # Prende sempre la prima ricetta
        top_healthiness_increments.append(top_recipe['healthiness_increment'])
        top_sustainability_increments.append(top_recipe['sustainability_increment'])
        top_sustainameal_score_increments.append(top_recipe['sustainameal_score_increment'])

# Calcola le medie degli incrementi per le prime ricette
average_top_healthiness_increment = sum(top_healthiness_increments) / len(top_healthiness_increments) if top_healthiness_increments else None
average_top_sustainability_increment = sum(top_sustainability_increments) / len(top_sustainability_increments) if top_sustainability_increments else None
average_top_sustainameal_score_increment = sum(top_sustainameal_score_increments) / len(top_sustainameal_score_increments) if top_sustainameal_score_increments else None

# Crea un DataFrame con una riga contenente le medie
top_results_df_10 = pd.DataFrame({
    'top_healthiness_increment_mean': [average_top_healthiness_increment],
    'top_sustainability_increment_mean': [average_top_sustainability_increment],
    'top_sustainameal_score_increment_mean': [average_top_sustainameal_score_increment],
    'gpt_rerank': [False],
    'k':10,
})

# Stampa il DataFrame dei risultati
print(top_results_df_10)


   top_healthiness_increment_mean  top_sustainability_increment_mean  \
0                       16.424224                            139.866   

   top_sustainameal_score_increment_mean  gpt_rerank   k  
0                             111.424199       False  10  


In [21]:
import json
import pandas as pd

# Carica i dati dal file JSON
with open('experiment_results_with_gpt_and_similarity_k_10.json', 'r') as file:
    experiment_data = json.load(file)

# Converti i dati in un DataFrame
experiment_df = pd.DataFrame(experiment_data)

# Filtra per gli esperimenti in cui GPT ha fatto una scelta
gpt_choices_df = experiment_df.dropna(subset=['gpt_choice'])

# Calcola le corrispondenze esatte
exact_matches = gpt_choices_df.apply(lambda row: row['ordered_by_sustainameal'][0]['title'] == row['gpt_choice'], axis=1)

# Calcola la percentuale di corrispondenze esatte
exact_match_count = exact_matches.sum()
total_choices = len(gpt_choices_df)
exact_match_percentage = (exact_match_count / total_choices * 100) if total_choices > 0 else 0

# Stampa il conteggio delle corrispondenze esatte e la percentuale
print(f"Corrispondenze esatte: {exact_match_count}")
print(f"Percentuale di corrispondenza esatta con GPT: {exact_match_percentage}%")


Corrispondenze esatte: 5
Percentuale di corrispondenza esatta con GPT: 7.462686567164178%


In [22]:
import json
import pandas as pd

# Assumi che 'SustainaMeal', 'acceptable_tags', 'nutrients_string', e 'recipe_titles' siano definiti

# Parametri per gli esperimenti
k_values = [1]
match_all_tags_values = [False]

# Funzione ausiliaria per convertire oggetti in formati serializzabili in JSON
def convert_to_serializable(obj):
    if hasattr(obj, 'to_dict'):
        return obj.to_dict('records')
    elif hasattr(obj, 'tolist'):
        return obj.tolist()
    elif isinstance(obj, (list, dict, str, int, float, bool, type(None))):
        return obj
    else:
        return str(obj)

results = []  # Lista per raccogliere i risultati degli esperimenti

for index, row in selected_recipes.iterrows():
    input_recipe = row['title']
    match_results = {}  # Dizionario per memorizzare i risultati di match per ogni k
    for k in k_values:
        match_results[k] = {'exact_match': False, 'match_count': 0}  # Inizializza il conteggio e il match esatto per ogni k
        for match_all_tags in match_all_tags_values:
            try:
                similar_recipes = sm.find_similar_recipes(input_recipe, k, acceptable_tags=acceptable_tags, match_all_tags=match_all_tags)
                ordered_recipes = sm.order_recipe_by_sustainameal()
                ordered_recipes_gpt = sm.order_recipe_by_sustainameal()
                result = {
                    'recipe_name': input_recipe,
                    'recipe_id': row['recipe_id'],
                    'sustnability_score': row['sustainability_score'],
                    'sustnability_label': row['sustainability_label'],
                    'who_score': row['who_score'],
                    'healthiness_label': row['healthiness_label'],
                    'k': k,
                    'nutrients': nutrients_string,
                    'match_all_tags': match_all_tags,
                    'similar_recipes': convert_to_serializable(similar_recipes) if similar_recipes is not None else None,
                    'ordered_by_sustainameal': convert_to_serializable(ordered_recipes) if ordered_recipes is not None else None
                }
                results.append(result)
            except Exception as e:
                print(f"An error occurred while finding similar recipes: {e}")
                best_choice_by_gpt = None



with open('experiment_results_with_gpt_and_similarity_k_1.json', 'w') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 54.88batch/s]

Tags to match: ['main-dish', 'beef', 'meat']
Number of filtered recipes: 28937


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 89.09batch/s]


Tags to match: ['main-dish', 'meat', 'pasta-rice-and-grains']
Number of filtered recipes: 34344


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 86.07batch/s]


...

Tags to match: ['main-dish', 'beef', 'meat']
Number of filtered recipes: 28937


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 84.18batch/s]


Tags to match: ['main-dish', 'pasta', 'pasta-rice-and-grains']
Number of filtered recipes: 30436


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 83.72batch/s]


Tags to match: ['main-dish', 'beef', 'rice', 'meat', 'pasta-rice-and-grains']
Number of filtered recipes: 34344


In [23]:
import json
import pandas as pd

# Carica i dati dal file JSON
with open('experiment_results_with_gpt_and_similarity_k_1.json', 'r') as file:
    experiment_data = json.load(file)

# Converti i dati in un DataFrame
experiment_df = pd.DataFrame(experiment_data)

# Inizializza i contatori per gli incrementi
top_healthiness_increments = []
top_sustainability_increments = []
top_sustainameal_score_increments = []

# Calcola gli incrementi per le prime ricette in ordered_by_sustainameal
for index, row in experiment_df.iterrows():
    if row['ordered_by_sustainameal']:  # Controlla se la lista non è vuota
        top_recipe = row['ordered_by_sustainameal'][0]  # Prende sempre la prima ricetta
        top_healthiness_increments.append(top_recipe['healthiness_increment'])
        top_sustainability_increments.append(top_recipe['sustainability_increment'])
        top_sustainameal_score_increments.append(top_recipe['sustainameal_score_increment'])

# Calcola le medie degli incrementi per le prime ricette
average_top_healthiness_increment = sum(top_healthiness_increments) / len(top_healthiness_increments) if top_healthiness_increments else None
average_top_sustainability_increment = sum(top_sustainability_increments) / len(top_sustainability_increments) if top_sustainability_increments else None
average_top_sustainameal_score_increment = sum(top_sustainameal_score_increments) / len(top_sustainameal_score_increments) if top_sustainameal_score_increments else None

# Crea un DataFrame con una riga contenente le medie
top_results_df_1 = pd.DataFrame({
    'top_healthiness_increment_mean': [average_top_healthiness_increment],
    'top_sustainability_increment_mean': [average_top_sustainability_increment],
    'top_sustainameal_score_increment_mean': [average_top_sustainameal_score_increment],
    'gpt_rerank': [False],
    'k':1,
})

# Stampa il DataFrame dei risultati
print(top_results_df_1)


   top_healthiness_increment_mean  top_sustainability_increment_mean  \
0                        7.588169                          76.526167   

   top_sustainameal_score_increment_mean  gpt_rerank  k  
0                               60.33972       False  1  


In [24]:
# Concatenazione dei DataFrame
final_results = pd.concat([gpt_results_df, top_results_df_10, top_results_df_1])

# Visualizzazione del DataFrame risultante
print(final_results)

   top_healthiness_increment_mean  top_sustainability_increment_mean  \
0                       15.833658                          90.176623   
0                       16.424224                         139.866000   
0                        7.588169                          76.526167   

   top_sustainameal_score_increment_mean  gpt_rerank   k  
0                              72.819732        True  10  
0                             111.424199       False  10  
0                              60.339720       False   1  


In [25]:
# Funzione per colorare i valori in base al segno
def color_negative_red_positive_green(value):
    if value < 0:
        return 'red'
    elif value > 0:
        return 'green'
    else:
        return 'black'

# Funzione per colorare i valori in base al segno e arrotondare i numeri
def apply_formatting(tbl, df):
    num_cols = df.shape[1]  # Ottiene il numero di colonne nel DataFrame
    for (i, j), cell in tbl.get_celld().items():
        # Salta le prime due colonne, l'intestazione, e l'ultima colonna
        if j == num_cols-1:
            continue
        val = cell.get_text().get_text()
        try:
            num = float(val)
            cell.get_text().set_text(f'{num:.2f}')
            cell.get_text().set_color('green' if num > 0 else 'red' if num < 0 else 'black')
        except ValueError:
            pass  # Non è un numero, non fare nulla



def abbreviate_column_names(df):
    abbreviations = {'top_healthiness_increment_mean': 'Top_H_inc_mean',
                     'top_sustainability_increment_mean': 'Top_S_inc_mean',
                     'top_sustainameal_score_increment_mean': 'Top_SM_inc_mean'}
    return df.rename(columns=abbreviations)


final_results = abbreviate_column_names(final_results)


In [26]:
final_results

,Top_H_inc_mean,Top_S_inc_mean,Top_SM_inc_mean,gpt_rerank,k
0,15.833658,90.176623,72.819732,True,10
0,16.424224,139.866000,111.424199,False,10
0,7.588169,76.526167,60.339720,False,1


In [27]:
import pandas as pd
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(10, 3))

ax.axis('off')

tbl = ax.table(
    cellText=final_results.values,
    colLabels=final_results.columns,
    loc='center',
    cellLoc='center',
)

apply_formatting(tbl, final_results)  # Applica il colore e l'arrotondamento qui

tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1.2, 1.2)

plt.show()
